In [41]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import re
import nltk
#nltk.download()  ## already downloaded : do I still need to load it?
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer  ## use this to create BAG of WORDS features 
from sklearn.ensemble import RandomForestClassifier


In [46]:
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

- Here, "header=0" indicates that the first line of the file contains column names,
- "delimiter=\t" indicates that the fields are separated by tabs, and
- quoting=3 tells Python to ignore doubled quotes, otherwise we may encounter errors trying to read the file.

In [47]:
train.shape  ## 25000,3
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


- id - Unique ID of each review
- sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
- review - Text of the review

In [5]:
train["review"][11100]

'"OK i own this DVD i got it new at amazon... i mean i think its badass and a pretty cool flick and melissa bale the slutty/bitchy girl they pick up is hot as hell ..., the acting sucks and the whole polt just sucks the clown is some huge guy wearing a mask and its disgusting but its OK i wouldn\'t recommend it if like u wanted to rent a good entertaining flick after a hard days work but if u have nothing else to do and ur obbsessed with this stupid movies like i am, watch it sometime, and i do not know how artisan DVD has S.I.C.K. in its DVD collection , s.i.c.k. is not good enough to be owned by a half way decent movie company OK well thats all"'

- There are HTML tags and we need to remove them to clean the data for machine learning algorithms. For this we use beautiful soup

In [6]:
# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(train["review"][0])  
print (train["review"][0])  ## raw text
print(example1.get_text())  ## processed text or cleaned text

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- Calling get_text() gives us the text without any HTML tags and markup languages.
- Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions

#### Kaggle tutorial
- There might be some expressions like !!! or :-) which might be important for sentiment analysis, but this is ignored at the moment
    - using regex to remove punctuations.
- We also remove numbers, but try out later with numbers as well. We can deal with them in two ways:
    1. treat them as words
    2. Replace with the placeholder string such as 'NUM' 

In [8]:
# Use regular expressions to do a find-and-replace

letters_only = re.sub("[^a-zA-Z]",    # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
#print(letters_only)
#convert the text into lower case

lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

- Finally, we need to decide how to deal with frequently occurring words that don't carry much meaning.
- Such words are called "stop words"; in English they include words such as "a", "and", "is", and "the". 

In [14]:
s = stopwords.words("english")
len(s)

179

In [20]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print(words) 

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

- There are many other things we could do to the data
    - For example, Porter Stemming and Lemmatizing (both available in NLTK) would allow us to treat "messages", "message", and "messaging" as the same word, which could certainly be useful. 

In [15]:
#summing up everything into a function

def review_to_words(raw_review) :
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()
    # 2. Remove non-letters (punctuations and number)
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    # 3. Lowercase the text
    lower_case = letters_only.lower()
    words = lower_case.split()
    # 4. Load the stopwords
    #    In python, searching a set is much faster than a list. 
    #    So let's convert the list to a set
    stops = set(stopwords.words("english"))     ## 
    # 5. Remove stopwords
    meaningful_words = [w for w in words if not w in stops]
    # 6. Join the words in one string separated by a space and
    #    return the result
    return(" ".join(meaningful_words))
    

<font color='red'>__Stopwords:__ this is not dependant on our dataset?, so why can't we just call it outside the function?</font>

In [21]:
## testing the function
clean_review = review_to_words(train["review"][0])
print(clean_review)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

##### Let's apply to the training dataset

In [29]:
# Get the number of reviews based on dataset column size
num_reviews = train["review"].size
# Initialise an empty list for clean_reviews
clean_train_reviews = []
# run the function on each review text

for i in range(0, num_reviews) : 
    clean_train_reviews.append(review_to_words(train["review"][i]))

- So we cleaned and pre-processed the data. 
- So, the question is how do we convert them to some kind of numerical representation for machine learning. 
    - Bag of words

##### Applying Bag of Words using random forest

In [36]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)
# Numpy arrays are easy to work with, so convert the result to an array

train_data_features = train_data_features.toarray()
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [37]:
train_data_features.shape

(25000, 5000)

- CountVectorizer comes with its own options to automatically do preprocessing, tokenization, and stop word removal
    - for each of these, instead of specifying "None", we could have used a built-in method or specified our own function to use.  
    - See the function documentation for more details. However, we wanted to write our own function for data cleaning in this tutorial to show you how it's done step by step. <font color='green'> __let's try with the vectorizer again !!!__ :) </font>

##### Now that the Bag of Words model is trained, let's look at the vocabulary:

In [39]:

vocab = vectorizer.get_feature_names()
vocab

5000

In [40]:
# We can also print the counts of each word in the vocabulary:

# Sums the count of each word
dist = np.sum(train_data_features, axis=0)
# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag) 

187 abandoned
125 abc
108 abilities
454 ability
1259 able
85 abraham
116 absence
83 absent
352 absolute
1485 absolutely
306 absurd
192 abuse
91 abusive
98 abysmal
297 academy
485 accent
203 accents
300 accept
130 acceptable
144 accepted
92 access
318 accident
200 accidentally
88 accompanied
124 accomplished
296 according
186 account
81 accuracy
284 accurate
123 accused
179 achieve
139 achieved
124 achievement
90 acid
971 across
1251 act
658 acted
6490 acting
3354 action
311 actions
83 activities
2389 actor
4486 actors
1219 actress
369 actresses
394 acts
793 actual
4237 actually
148 ad
302 adam
98 adams
453 adaptation
80 adaptations
154 adapted
810 add
439 added
166 adding
347 addition
337 adds
113 adequate
124 admire
621 admit
134 admittedly
101 adorable
510 adult
376 adults
100 advance
90 advanced
153 advantage
510 adventure
204 adventures
91 advertising
259 advice
90 advise
346 affair
93 affect
113 affected
104 afford
126 aforementioned
343 afraid
212 africa
255 african
187 afternoon

186 dragon
130 drags
89 drake
1411 drama
141 dramas
667 dramatic
198 draw
114 drawing
428 drawn
118 draws
240 dreadful
663 dream
436 dreams
87 dreary
81 dreck
181 dress
293 dressed
86 dressing
245 drew
148 drink
175 drinking
448 drive
125 drivel
239 driven
195 driver
155 drives
269 driving
205 drop
131 dropped
85 dropping
102 drops
403 drug
325 drugs
292 drunk
119 drunken
230 dry
96 dub
224 dubbed
153 dubbing
83 dud
183 dude
909 due
132 duke
816 dull
609 dumb
121 duo
91 dust
104 dutch
106 duty
2345 dvd
314 dying
112 dynamic
94 eager
86 ear
87 earl
662 earlier
1605 early
100 earned
99 ears
928 earth
110 ease
132 easier
892 easily
170 east
83 eastern
138 eastwood
802 easy
275 eat
90 eaten
278 eating
109 eccentric
341 ed
310 eddie
95 edgar
441 edge
82 edgy
107 edie
262 edited
774 editing
89 edition
119 editor
97 education
83 educational
204 edward
141 eerie
633 effect
512 effective
187 effectively
2204 effects
792 effort
254 efforts
128 ego
221 eight
101 eighties
1866 either
118 elaborate

188 julia
86 julian
174 julie
300 jump
92 jumped
125 jumping
160 jumps
90 june
188 jungle
91 junior
190 junk
418 justice
98 justify
89 justin
110 juvenile
135 kane
79 kansas
114 kapoor
115 karen
148 karloff
300 kate
91 kay
308 keaton
1601 keep
276 keeping
642 keeps
88 keith
429 kelly
123 ken
108 kennedy
116 kenneth
750 kept
289 kevin
425 key
99 khan
264 kick
98 kicked
92 kicking
133 kicks
1199 kid
109 kidding
128 kidnapped
1844 kids
1234 kill
1111 killed
1455 killer
245 killers
694 killing
132 killings
530 kills
209 kim
2783 kind
275 kinda
191 kinds
999 king
95 kingdom
114 kirk
171 kiss
83 kissing
118 kitchen
897 knew
147 knife
139 knock
6166 know
447 knowing
283 knowledge
1080 known
901 knows
78 kolchak
270 kong
131 korean
130 kubrick
128 kudos
80 kumar
243 kung
83 kurosawa
149 kurt
96 kyle
552 la
131 lab
1058 lack
121 lacked
277 lacking
79 lackluster
365 lacks
295 ladies
848 lady
160 laid
250 lake
742 lame
362 land
81 landing
115 landscape
86 landscapes
170 lane
529 language
556 larg

2733 rather
928 rating
166 ratings
87 rats
82 rave
176 raw
378 ray
92 raymond
123 rd
246 reach
115 reached
102 reaches
95 reaching
108 react
249 reaction
136 reactions
1964 read
80 reader
685 reading
93 reads
336 ready
4737 real
126 realise
279 realism
757 realistic
987 reality
654 realize
317 realized
199 realizes
98 realizing
11736 really
2323 reason
117 reasonable
119 reasonably
596 reasons
103 rebel
226 recall
112 receive
262 received
83 receives
510 recent
579 recently
90 recognition
195 recognize
113 recognized
1667 recommend
489 recommended
283 record
104 recorded
85 recording
819 red
326 redeeming
143 redemption
118 reduced
163 reed
88 reel
166 reference
249 references
92 reflect
81 reflection
206 refreshing
78 refused
154 refuses
166 regard
174 regarding
125 regardless
189 regret
266 regular
86 reid
235 relate
202 related
94 relation
102 relations
966 relationship
361 relationships
126 relative
213 relatively
89 relatives
807 release
986 released
132 relevant
242 relief
106 re

##### Using Random Forest
- At this point, we have numeric training features from the Bag of Words and the original sentiment labels for each feature vector, so let's do some supervised learning! 

In [44]:
#Initialise the random forest with 100 trees
forest = RandomForestClassifier(n_estimators=100)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
forest = forest.fit(train_data_features, train["sentiment"])

##### Now let's run it on the test data

In [50]:
#Load the test data

test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3 )
test.shape


In [53]:
# Cleaning and pre-processing the test data

num_train_reviews = test["review"].size
clean_test_reviews = []

for j in range(0, num_train_reviews):
    clean_test_reviews.append(review_to_words(test["review"][j]))


In [54]:
# Get a bag of words for the train set and convert them into a numpy array

test_data_features = vectorizer.fit_transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
test_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [55]:
# Predict the sentiment for the test data
sentiment_pred = forest.predict(test_data_features)
sentiment_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [59]:
# Copy the result to a dataframe

output = pd.DataFrame(data={"id":test["id"], "sentiment":sentiment_pred, "review":test["review"]})
pd.set_option('display.max_colwidth', -1)
output

,id,sentiment,review
0,"""12311_10""",0,"""Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce's short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."""
1,"""8348_2""",0,"""This movie is a disaster within a disaster film. It is full of great action scenes, which are only meaningful if you throw away all sense of reality. Let's see, word to the wise, lava burns you; steam burns you. You can't stand next to lava. Diverting a minor lava flow is difficult, let alone a significant one. Scares me to think that some might actually believe what they saw in this movie.<br /><br />Even worse is the significant amount of talent that went into making this film. I mean the acting is actually very good. The effects are above average. Hard to believe somebody read the scripts for this and allowed all this talent to be wasted. I guess my suggestion would be that if this movie is about to start on TV ... look away! It is like a train wreck: it is so awful that once you know what is coming, you just have to watch. Look away and spend your time on more meaningful content."""
2,"""5828_4""",0,"""All in all, this is a movie for kids. We saw it tonight and my child loved it. At one point my kid's excitement was so great that sitting was impossible. However, I am a great fan of A.A. Milne's books which are very subtle and hide a wry intelligence behind the childlike quality of its leading characters. This film was not subtle. It seems a shame that Disney cannot see the benefit of making movies from more of the stories contained in those pages, although perhaps, it doesn't have the permission to use them. I found myself wishing the theater was replaying \""Winnie-the-Pooh and Tigger too\"", instead. The characters voices were very good. I was only really bothered by Kanga. The music, however, was twice as loud in parts than the dialog, and incongruous to the film.<br /><br />As for the story, it was a bit preachy and militant in tone. Overall, I was disappointed, but I would go again just to see the same excitement on my child's face.<br /><br />I liked Lumpy's laugh...."""
3,"""7186_2""",1,"""Afraid of the Dark left me with the impression that several different screenplays were written, all too short for a feature length film, then spliced together clumsily into this Frankenstein's monster.<br /><br />At his best, the protagonist, Lucas, is creepy. As hard as it is to draw a bead on the secondary characters, they're far more sympathetic.<br /><br />Afraid of the Dark could have achieved mediocrity had it taken just one approach and seen it through -- and had it made Lucas simply psychotic and confused instead of ghoulish and off-putting. I wanted to see him packed off into an asylum so the rest of the characters could have a normal life."""
4,"""12128_7""",0,"""A very accurate depiction of small time mob life filmed in New Jersey. The story, characters and script are believable but the acting drops the ball. Still, it's worth watching, especially for the strong images, some still with me even though I first viewed this 25 years ago.<br /><br />A young hood steps up and starts doing bigger things (tries to) but these things keep going wrong, leading the local boss to suspect that his end is being skimmed off, not a good place to be if you enjoy your health, or life.<br /><br />This is the film that i